In [11]:
import io
import base64
import os
from cellpose import models
import numpy as np
import pandas as pd
from skimage.measure import regionprops
from PIL import Image
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# ---------- Helper functions ----------

def read_image_from_bytes(b):
    img = Image.open(io.BytesIO(b)).convert('RGB')
    return np.asarray(img)

def compute_radii(mask):
    props = regionprops(mask)
    return pd.DataFrame([{'radius_px': np.sqrt(p.area / np.pi)} for p in props])

def draw_segmentation(image, mask):
    from skimage import color
    outlined = color.label2rgb(mask, image, kind='overlay', bg_label=0, alpha=0.4)
    fig, ax = plt.subplots(figsize=(6,6))
    ax.imshow(outlined)
    ax.axis('off')
    return fig

def make_download_link_bytesio(bytes_io, filename, mimetype):
    bytes_io.seek(0)
    b64 = base64.b64encode(bytes_io.read()).decode()
    href = f'<a download="{filename}" href="data:{mimetype};base64,{b64}">{filename}</a>'
    return HTML(href)

# ---------- UI ----------

intro = widgets.HTML("""
<h3>CellDivision: GUVs analysis Tool</h3>
<p>You are in the GUVs analysis data processing page! Here, you can load your image and analyse it with the CellDivision segmentation tool.</p>
<p>Visit the documentation to see the manual for info on how the tool works.</p>
<p><b>Click here to return to the index and select a different application.</b></p>
<p><i>NOTE: Loading the widgets may require some seconds.</i></p>
""")

uploader = widgets.FileUpload(accept='image/*', multiple=False)
run_button = widgets.Button(description='Run segmentation', button_style='success')
status_label = widgets.HTML("")
output = widgets.Output()

# ---------- Load Cellpose model once ----------

model_output = widgets.Output()
with model_output:
    print("Loading Cellpose model (cyto)... please wait.")
model = models.Cellpose(model_type='cyto', gpu=False)
with model_output:
    clear_output()
    print("✅ Model loaded and ready.")

# ---------- Main function ----------

def on_run_click(b):
    with output:
        clear_output()
    status_label.value = "<p style='color:orange;'>⏳ Processing... please wait.</p>"

    if len(uploader.value) == 0:
        status_label.value = "<p style='color:red;'>⚠️ Please upload an image first.</p>"
        return

    uploaded_file = uploader.value[0]
    filename = uploaded_file['name']
    content = uploaded_file['content']

    # 🔹 Rimuovi estensione dal nome file
    base_name, _ = os.path.splitext(filename)

    try:
        image = read_image_from_bytes(content)
    except Exception as e:
        status_label.value = f"<p style='color:red;'>Error reading image: {e}</p>"
        return

    try:
        masks, _, _, _ = model.eval([image], diameter=None, channels=[0, 0])
    except Exception as e:
        status_label.value = f"<p style='color:red;'>Segmentation error: {e}</p>"
        return

    mask = masks[0]
    df = compute_radii(mask)

    status_label.value = f"<p style='color:green;'>✅ Segmentation complete. Detected {len(df)} cells.</p>"

    # 🔹 Salva CSV con nome senza estensione
    csv_buffer = io.BytesIO()
    df.to_csv(csv_buffer, index=False)
    with output:
        clear_output()
        display(make_download_link_bytesio(csv_buffer, base_name + '_radii.csv', 'text/csv'))

        fig = draw_segmentation(image, mask)
        plt.show()
        plt.close(fig)

run_button.on_click(on_run_click)

# ---------- Layout ----------

app = widgets.VBox([
    intro,
    model_output,
    uploader,
    run_button,
    status_label,
    output
])

display(app)
